In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 4
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 1e6
length = 30

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
#n_select = 10
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e

    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1000000.00, score:-57.32
 Observe Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:13.2, cum_e:13.2, new_i:11.8, cum_i:11.8
 True State Time: 1 | S: 12563950.0, E: 37.0, A: 0.0, I: 11.8, D: 0.0, R: 1.3, Budget: 1000000.00, score:-138.59
 Observe Time: 1 | S: 12563949.0, E: 36.0, A: 0.0, I: 11.8, D: 0.0, R: 0.0
 Time: 1 | new_e:18.6, cum_e:31.8, new_i:16.3, cum_i:28.2
 True State Time: 2 | S: 12563919.0, E: 49.9, A: 0.0, I: 28.1, D: 0.0, R: 2.9, Budget: 1000000.00, score:-254.10
 Observe Time: 2 | S: 12563919.0, E: 50.0, A: 0.0, I: 28.1, D: 0.0, R: 0.0
 Time: 2 | new_e:26.2, cum_e:58.0, new_i:22.8, cum_i:51.0
 True State Time: 3 | S: 12563875.3, E: 68.5, A: 0.0, I: 50.9, D: 0.0, R: 5.2, Budget: 1000000.00, score:-418.56
 Observe Time: 3 | S: 12563877.0, E: 68.0, A: 0.0, I: 50.9, D: 0.0, R: 3.0
 Time: 3 | new_e:37.2, cum_e:95.2, new_i:32.1, cum_i:83.1
 True State Time: 4

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 25 | S: 11931779.0, E: 1.8, A: 100.6, I: 286.7, D: 1.9, R: 631830.0, Budget: 533324.29, score:-3983.94
 Observe Time: 25 | S: 11931806.0, E: 0.0, A: 100.6, I: 286.7, D: 1.9, R: 631827.0
 Time: 25 | new_e:27.4, cum_e:891.1, new_i:29.2, cum_i:1159.2
 True State Time: 26 | S: 11917395.7, E: 28.7, A: 85.5, I: 293.8, D: 2.0, R: 646194.3, Budget: 503455.23, score:-4041.51
 Observe Time: 26 | S: 11917425.0, E: 26.0, A: 85.5, I: 293.8, D: 2.0, R: 646192.0
 Time: 26 | new_e:12.8, cum_e:903.9, new_i:20.8, cum_i:1180.0
 True State Time: 27 | S: 11881239.7, E: 12.8, A: 96.8, I: 277.8, D: 2.1, R: 682370.9, Budget: 473623.94, score:-4068.16
 Observe Time: 27 | S: 11881270.0, E: 11.0, A: 96.8, I: 277.8, D: 2.1, R: 682368.0
 Time: 27 | new_e:5.7, cum_e:909.6, new_i:15.5, cum_i:1195.5
 True State T

 True State Time: 55 | S: 11258045.8, E: 143.8, A: 36.0, I: 208.0, D: 2.6, R: 1305563.8, Budget: 183701.86, score:-5831.36
 Observe Time: 55 | S: 11258635.0, E: 0.0, A: 36.0, I: 208.0, D: 2.6, R: 1305539.0
 Time: 55 | new_e:60.5, cum_e:1309.9, new_i:59.6, cum_i:1663.2
 True State Time: 56 | S: 11242278.9, E: 172.0, A: 40.9, I: 241.6, D: 2.6, R: 1321264.1, Budget: 176098.90, score:-6146.16
 Observe Time: 56 | S: 11242968.0, E: 0.0, A: 40.9, I: 241.6, D: 2.6, R: 1321235.0
 Time: 56 | new_e:72.2, cum_e:1382.1, new_i:71.0, cum_i:1734.2
 True State Time: 57 | S: 11227614.4, E: 206.4, A: 46.2, I: 282.3, D: 2.6, R: 1335848.1, Budget: 168881.54, score:-6523.33
 Observe Time: 57 | S: 11228422.0, E: 0.0, A: 46.2, I: 282.3, D: 2.6, R: 1335814.0
 Time: 57 | new_e:86.6, cum_e:1468.7, new_i:84.8, cum_i:1818.9
    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.14.6\x3a Maximum Number of Iterations
        Exceeded.
 True State Time: 58 | S: 112

 True State Time: 81 | S: 11130999.9, E: 232.4, A: 458.9, I: 1641.6, D: 10.2, R: 1430657.1, Budget: 1895.27, score:-16810.60
 Observe Time: 81 | S: 11135134.0, E: 29.0, A: 458.9, I: 1641.6, D: 10.2, R: 1430474.0
 Time: 81 | new_e:166.8, cum_e:3713.4, new_i:185.3, cum_i:4574.1
 True State Time: 82 | S: 11138670.0, E: 363.7, A: 388.9, I: 1825.0, D: 12.0, R: 1422740.4, Budget: 1850.99, score:-17588.09
 Observe Time: 82 | S: 11142943.0, E: 108.0, A: 388.9, I: 1825.0, D: 12.0, R: 1422548.0
 Time: 82 | new_e:151.6, cum_e:3865.0, new_i:181.7, cum_i:4755.9
 True State Time: 83 | S: 11146321.4, E: 459.9, A: 329.6, I: 2004.8, D: 14.0, R: 1414870.5, Budget: 1804.31, score:-18551.59
 Observe Time: 83 | S: 11150774.0, E: 134.0, A: 329.6, I: 2004.8, D: 14.0, R: 1414666.0
 Time: 83 | new_e:191.8, cum_e:4056.8, new_i:212.5, cum_i:4968.3
 True State Time: 84 | S: 11153862.1, E: 581.5, A: 279.3, I: 2215.0, D: 16.2, R: 1407046.0, Budget: 1754.42, score:-19749.51
 Observe Time: 84 | S: 11158542.0, E: 171.

In [7]:
(time.time() - tic) / ( T - start_time + 1)


13.265983068241793